In [1]:
#Import dependencies
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

In [2]:
title = "Software Developer"  # Job title
location = "Linköping"  # Job location
start = 25  # Starting point for pagination

In [3]:
from collections import defaultdict
programming_languages = pd.read_csv('./skills.csv')['skills'].to_list()
programming_languages = set(programming_languages)

users_of_languages = defaultdict(int)
for l in programming_languages:
    users_of_languages[l] = 0

In [4]:
list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&start={start}"

response = requests.get(list_url)

list_data = response.text
list_soup = BeautifulSoup(list_data, "html.parser")
page_jobs = list_soup.find_all("li")
    
    

In [5]:
#Create an empty list to store the job postings
id_list = []

In [6]:
#Itetrate through job postings to find job ids
for job in page_jobs:
    base_card_div = job.find("div", {"class": "base-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    print(job_id)
    id_list.append(job_id)
    

3831016208
4333868335
4323225830
4336231021
4308232414
4318315576
4100013391
4008272815
4337151023
4338144855


In [7]:
job_list = []

for job_id in id_list:
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    
    job_response = requests.get(job_url)
    print(job_response.status_code)
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    
    job_post = {}
    
    try:
        job_post["job_title"] = job_soup.find("h2", {"class":"top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        job_post["job_title"] = None
        
    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["company_name"] = None
        
    try:
        job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job_post["time_posted"] = None
        
    try:
        job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
    except:
        job_post["num_applicants"] = None

    for lang in programming_languages:
        # description is in show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden
        if f" {lang.lower()}" in job_soup.find("div", {"class": "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden"}).text.lower():  
            users_of_languages[lang] += 1
            
        
    job_list.append(job_post)
    
    

200
200
200
200
200
200
200
200
200
200


In [8]:
# Create a pandas DataFrame using the list of job dictionaries 'job_list'
jobs_df = pd.DataFrame(job_list)
jobs_df

,job_title,company_name,time_posted,num_applicants
0,System Developer,B3 Reach,2 weeks ago,111 applicants
1,Fullstack Utvecklare & Arkitekter till Product...,Saab,2 weeks ago,None
2,Embedded Software Developer,Softube AB,None,None
3,Juniora utvecklare till nästa generations moln...,Framtiden AB,5 days ago,None
4,"Systems Engineer, Staff - Qualcomm, Linkoping",Qualcomm,6 days ago,101 applicants
5,Senior Fullstack-utvecklare,Rasulson Consulting AB,4 weeks ago,None
6,Junior Mjukvaruutvecklare inom inbyggda system,Sectra,1 week ago,None
7,"""Follow The Sun"" Tech Student Program",Sectra,1 week ago,None
8,AI Engineer - Autonomous systems,Saab,None,None
9,Utvecklare C#/.Net Norrköping,Castra,1 week ago,None


In [9]:
lang_usage = pd.DataFrame.from_dict(users_of_languages, orient='index', columns=['Number of Job Postings']).reset_index().rename(columns={'index': 'Programming Language'})
lang_usage.drop(lang_usage[lang_usage['Number of Job Postings'] == 0].index, inplace=True)
lang_usage = lang_usage.sort_values(by='Number of Job Postings', ascending=False)
lang_usage = lang_usage.reset_index(drop=True)
lang_usage

,Programming Language,Number of Job Postings
0,C,10
1,C#,5
2,Python,5
3,Go,5
4,C++,3
5,Linux,3
6,C/C++,3
7,Java,2
8,AWS,2
9,React,2
